In [ ]:
#Main information source: https://agromonitoring.com/api/polygons
import json
import requests
import geojson
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

token = ''

# Create a polygon

In [ ]:
#Fill out your polygon coordinates here
geometry = {"type":"Polygon","coordinates":[[[-0.48786,39.63892],[-0.48786,39.66084],[-0.44512,39.66084],[-0.44512,39.63892],[-0.48786,39.63892]]]}

In [ ]:
response_creation = requests.get(
    url = f'http://api.agromonitoring.com/agro/1.0/polygons?appid={token}')

In [ ]:
response_creation.json()

# Search for satellite data in the selected polygon

In [ ]:
from datetime import datetime
from datetime import timezone

In [ ]:
polygonID = '6059be4389177074678eda48'

#Start date of the data search (unix time, UTC)
start = datetime(2018, 10, 19, tzinfo=timezone.utc).timestamp()

#End date of the data search (unix time, UTC)
end = datetime(2020, 10, 19, tzinfo=timezone.utc).timestamp()



In [ ]:
property = 'mean'

In [ ]:
#Here you can select further parameters, such as satellite, resolution, etc. (https://agromonitoring.com/api/images)
response = requests.get(
    url = f'http://api.agromonitoring.com/agro/1.0/image/search?start={start}&end={end}&polyid={polygonID}&appid={token}'
)

In [ ]:
response2 = requests.get(url = response.json()[0]['stats']['ndvi'])

In [ ]:
#Retrieve the time of each data collection
time_points = [datetime.utcfromtimestamp(t1['dt']) for t1 in response.json()]

In [ ]:
#mean_ndvi_points = [requests.get(url = elem['stats']['ndvi']).json()[property] for elem in response.json() if property in requests.get(url = elem['stats']['ndvi']).json()]

In [ ]:
#Retrieve the statistical values
ndvi_points2 = pd.DataFrame()
for ind in range(0,len(response.json())):
    response2 = requests.get(url = response.json()[ind]['stats']['ndvi'])
    if 'mean' in response2.json():
        ndvi_points2 = ndvi_points2.append([response2.json()[property]])
    else:
        ndvi_points2 = ndvi_points2.append([np.nan])


In [ ]:
data1 ={'Year-Month-Day': time_points, 
        f'{property} NDVI': ndvi_points2[0] }
df = pd.DataFrame(data=data1)

In [ ]:
df

In [ ]:
#Font for plot
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 14}

plt.rc('font', **font)

In [ ]:
plt.figure(figsize=(10,6))
g = sns.lineplot(data=df, x='Year-Month-Day', y=f'{property} NDVI', markers=True, dashes=False)

g.set_xlabel('Year-Month-Day', fontweight='bold', fontsize=18)
g.set_ylabel(f'{property} NDVI', fontweight='bold', fontsize=18)

# labels appear outside of plot area, so auto-adjust
plt.tight_layout()

In [ ]:
response.json()[3]['tile']['ndvi']

# Soil data

In [ ]:
#Current soil data is available for free accounts. Historical data is not. However, a user can log current data.
response = requests.get(
    url = f'http://api.agromonitoring.com/agro/1.0/soil?polyid={polygonID}&appid={token}')

In [ ]:
#Response is with 'dt' = timestamp, 't10' = Temperature in Kelvin 10 cm below surface, 
#'moisture' = soil moisture in m3/m3, 't0' = Surface Temperature in Kelvin
response.json()